In [1]:
import os.path as osp
import sys
path = osp.dirname(osp.abspath(''))
sys.path.append(path)
path

'/mnt/luoyz_dair/OpenBioMed'

In [34]:
import numpy as np
from tqdm import tqdm

from open_biomed.datasets.dp_dataset import MoleculeNetDataset

config = {
    "mol": {
        "modality": ["structure"],
        "featurizer": {
            "structure": {
                "name": "fingerprint",
                "which": "rdk",
                "radius": 2,
                "fp_size": 256,
                "return_type": "np"
            }
        }
    }
}
path = "../datasets/dp"
dataset = MoleculeNetDataset(path, config, name="MUV")
dataset._configure_featurizer()
dataset.drugs = [dataset.drug_featurizer(drug) for drug in tqdm(dataset.drugs)]

train = dataset.index_select(dataset.train_index)
val = dataset.index_select(dataset.val_index)
test = dataset.index_select(dataset.test_index)
train_X, train_Y = np.array(train.drugs), np.array(train.labels)
val_X, val_Y = np.array(val.drugs), np.array(val.labels)
test_X, test_Y = np.array(test.drugs), np.array(test.labels)
print(train_X.shape, val_X.shape, test_X.shape)

100%|██████████| 93087/93087 [01:37<00:00, 957.05it/s] 


(74469, 256) (9309, 256) (9309, 256)


In [41]:
from sklearn.ensemble import RandomForestClassifier
from open_biomed.utils.metrics import roc_auc

"""
params_dict = {
    "n_estimators": [50, 100, 200],
    "max_features": ["sqrt", "log2", None],
}
results = []
"""
params_dict = {
    "n_estimators": [200],
    "max_features": [None]
}

best_val = 0
best_test = None

for n_estimators in params_dict["n_estimators"]:
    for max_features in params_dict["max_features"]:
        cur_val = []
        cur_test = []
        for seed in (2, 3, 4):
            np.random.seed(seed)
            model = RandomForestClassifier(
                n_estimators=n_estimators,
                max_features=max_features,
                n_jobs=4,
                max_depth=5
            )
            #model.fit(train_X, train_Y)
            val_per_task = []
            test_per_task = []
            print(train_Y.shape)
            for i in range(train_Y.shape[1]):
                print(np.sum(train_Y[:, i]), np.sum(val_Y[:, i]), np.sum(test_Y[:, i]))
                continue
                if train_Y.shape[1] != 1:
                    pred_val = model.predict_proba(val_X)[i][:, 1]
                    pred_test = model.predict_proba(test_X)[i][:, 1]
                else:
                    pred_val = model.predict_proba(val_X)[:, 1]
                    pred_test = model.predict_proba(test_X)[:, 1]
                #print("n_estimators = %d" % n_estimators)
                #print("max_features = ", max_features)
                val_per_task.append(roc_auc(val_Y[:, i], pred_val))
                test_per_task.append(roc_auc(test_Y[:, i], pred_test))
                #print("AUROC val = %.4lf" % cur_val)
                #print("AUROC_test = %.4lf" % cur_test)
            cur_val.append(val_per_task)
            cur_test.append(test_per_task)
        if np.mean(cur_val) > best_val:
            best_val = np.mean(cur_val)
            best_test = (np.mean(cur_test), np.mean([np.sqrt(np.var(np.array(cur_test)[:, i])) for i in range(train_Y.shape[1])]))
print("Results = %.4lf±%.4lf" % best_test)


(74469, 17)
22.0 5.0 0.0
21.0 2.0 6.0
20.0 3.0 7.0
23.0 4.0 3.0
18.0 4.0 7.0
25.0 2.0 2.0
25.0 3.0 2.0
21.0 3.0 4.0
24.0 3.0 2.0
21.0 3.0 4.0
21.0 2.0 6.0
23.0 2.0 4.0
24.0 6.0 0.0
26.0 2.0 2.0
22.0 4.0 3.0
23.0 4.0 2.0
21.0 0.0 3.0
(74469, 17)
22.0 5.0 0.0
21.0 2.0 6.0
20.0 3.0 7.0
23.0 4.0 3.0
18.0 4.0 7.0
25.0 2.0 2.0
25.0 3.0 2.0
21.0 3.0 4.0
24.0 3.0 2.0
21.0 3.0 4.0
21.0 2.0 6.0
23.0 2.0 4.0
24.0 6.0 0.0
26.0 2.0 2.0
22.0 4.0 3.0
23.0 4.0 2.0
21.0 0.0 3.0
(74469, 17)
22.0 5.0 0.0
21.0 2.0 6.0
20.0 3.0 7.0
23.0 4.0 3.0
18.0 4.0 7.0
25.0 2.0 2.0
25.0 3.0 2.0
21.0 3.0 4.0
24.0 3.0 2.0
21.0 3.0 4.0
21.0 2.0 6.0
23.0 2.0 4.0
24.0 6.0 0.0
26.0 2.0 2.0
22.0 4.0 3.0
23.0 4.0 2.0
21.0 0.0 3.0


/home/luoyz/.conda/envs/biomedgpt/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/luoyz/.conda/envs/biomedgpt/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


TypeError: must be real number, not NoneType

In [35]:
from sklearn.svm import SVC, LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from open_biomed.utils.metrics import roc_auc

cur_val = []
cur_test = []
for seed in [22]:
    val_per_task = []
    test_per_task = []
    for i in range(train_Y.shape[1]):
        print(seed, i)
        """
        model = SVC(
            kernel='linear', 
            C=0.05,
            random_state=seed,
            probability=True,
            #max_iter=1000
        )
        """

        model = CalibratedClassifierCV(LinearSVC(
            C=0.1,
            random_state=seed,
            dual='auto'
        ))

        model.fit(train_X, train_Y[:, i])
        
        pred_val = model.predict_proba(val_X)[:, 1]
        pred_test = model.predict_proba(test_X)[:, 1]
        auc_val = roc_auc(val_Y[:, i], pred_val)
        auc_test = roc_auc(test_Y[:, i], pred_test)
        if not np.isnan(auc_val) and not np.isnan(auc_test):
            val_per_task.append(auc_val)
            test_per_task.append(auc_test)
            #print(test_per_task[-1])
    cur_val.append(val_per_task)
    cur_test.append(test_per_task)
print(np.mean(cur_val))
print("Results = %.4lf±%.4lf" % (np.mean(cur_test), np.mean([np.sqrt(np.var(np.array(cur_test)[:, i])) for i in range(len(cur_test[0]))])))

22 0


/home/luoyz/.conda/envs/biomedgpt/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1132: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


22 1
22 2
22 3
22 4
22 5
22 6
22 7
22 8
22 9
22 10
22 11
22 12


/home/luoyz/.conda/envs/biomedgpt/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1132: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


22 13
22 14
22 15
22 16
0.6833759479380893
Results = 0.6412±0.0000


/home/luoyz/.conda/envs/biomedgpt/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1132: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


In [43]:
from scipy.stats import ttest_rel
a = [64.4, 66.0, 67.5]
b = [62.8, 64.0, 65.0]
print(ttest_rel(a, b))

TtestResult(statistic=7.810249675906737, pvalue=0.016001032391817542, df=2)
